In [3]:
# Importando as bibliotecas necessárias
import pandas as pd
import pyodbc
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
# Isso garante que podemos pegar nossa string de conexão segura
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")

print("Conectando ao banco de dados...")

# Query SQL para buscar os dados que nos interessam.
# Note o JOIN para combinar dados das tarefas com o status do projeto.
sql_query = """
SELECT 
    t.Descricao,
    t.DataInicioPrevista,
    t.DataFimPrevista,
    DATEDIFF(day, t.DataInicioPrevista, t.DataFimPrevista) AS DuracaoPrevista,
    t.DataInicioReal,
    t.DataFimReal,
    DATEDIFF(day, t.DataInicioReal, t.DataFimReal) AS DuracaoReal,
    p.Status AS StatusProjeto,
    t.Status AS StatusTarefa -- Esta é a nossa variável ALVO (o que queremos prever)
FROM 
    Tarefas t
JOIN 
    Projetos p ON t.ProjetoID = p.ProjetoID;
"""

try:
    # Conecta ao banco e usa o pandas para executar a query e carregar os dados em um DataFrame
    conn = pyodbc.connect(DATABASE_URL)
    df = pd.read_sql_query(sql_query, conn)
    conn.close()
    
    print("Dados carregados com sucesso!")
    
    # Mostra as 5 primeiras linhas do nosso conjunto de dados
    print("Amostra dos dados:")
    display(df.head())

except Exception as e:
    print(f"Ocorreu um erro: {e}")

Conectando ao banco de dados...


C:\Users\alenc\AppData\Local\Temp\ipykernel_18012\1520550302.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, conn)


Dados carregados com sucesso!
Amostra dos dados:


,Descricao,DataInicioPrevista,DataFimPrevista,DuracaoPrevista,DataInicioReal,DataFimReal,DuracaoReal,StatusProjeto,StatusTarefa
0,Deleniti inventore incidunt.,2025-11-21 21:12:51.543,2025-12-10 21:12:51.543,19,2025-11-21 21:12:51.543,2025-12-10 21:12:51.543,19,Em Andamento,Concluída no Prazo
1,Eum culpa nobis aliquid at.,2025-10-24 21:12:51.823,2025-11-16 21:12:51.823,23,2025-10-27 21:12:51.823,2025-11-20 21:12:51.823,24,Em Andamento,Atrasada
2,Occaecati culpa explicabo cumque deleniti elig...,2025-08-04 21:12:52.100,2025-08-13 21:12:52.100,9,2025-08-07 21:12:52.100,2025-08-22 21:12:52.100,15,Em Andamento,Atrasada
3,Vitae modi placeat qui soluta ea quos.,2025-08-26 21:12:52.380,2025-09-19 21:12:52.380,24,2025-08-27 21:12:52.380,2025-09-19 21:12:52.380,23,Em Andamento,Concluída no Prazo
4,Nam sapiente corrupti laborum ullam laboriosam.,2025-11-12 21:12:52.657,2025-11-29 21:12:52.657,17,2025-11-12 21:12:52.657,2025-11-29 21:12:52.657,17,Em Andamento,Concluída no Prazo


In [4]:
# Célula 2: Análise Exploratória dos Dados

print("## Informações Gerais do DataFrame ##")
df.info()

print("\n## Análise Estatística das Colunas Numéricas ##")
display(df.describe())

print("\n## Contagem do Nosso Alvo (Status da Tarefa) ##")
display(df['StatusTarefa'].value_counts())

## Informações Gerais do DataFrame ##
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1421 entries, 0 to 1420
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Descricao           1421 non-null   object        
 1   DataInicioPrevista  1421 non-null   datetime64[ns]
 2   DataFimPrevista     1421 non-null   datetime64[ns]
 3   DuracaoPrevista     1421 non-null   int64         
 4   DataInicioReal      1421 non-null   datetime64[ns]
 5   DataFimReal         1421 non-null   datetime64[ns]
 6   DuracaoReal         1421 non-null   int64         
 7   StatusProjeto       1421 non-null   object        
 8   StatusTarefa        1421 non-null   object        
dtypes: datetime64[ns](4), int64(2), object(3)
memory usage: 100.0+ KB

## Análise Estatística das Colunas Numéricas ##


,DataInicioPrevista,DataFimPrevista,DuracaoPrevista,DataInicioReal,DataFimReal,DuracaoReal
count,1421,1421,1421.000000,1421,1421,1421.000000
mean,2025-09-07 16:23:31.014894848,2025-09-24 00:47:09.734106880,16.349754,2025-09-09 05:07:35.912854528,2025-09-25 07:42:38.657400576,16.107671
min,2025-06-11 21:13:25.853000,2025-06-14 21:15:56.253000,3.000000,2025-06-11 21:14:46.753000,2025-06-15 21:17:18.340000,1.000000
25%,2025-07-23 21:15:38.092999936,2025-08-08 21:13:38.020000,10.000000,2025-07-25 21:13:38.292999936,2025-08-08 21:18:32.830000128,10.000000
50%,2025-09-07 21:16:48.600000,2025-09-23 21:15:14.903000064,16.000000,2025-09-08 21:19:39.492999936,2025-09-25 21:15:09.392999936,16.000000
75%,2025-10-22 21:15:22.903000064,2025-11-09 21:13:04.303000064,23.000000,2025-10-23 21:19:48.040000,2025-11-09 21:15:56.803000064,22.000000
max,2025-12-07 21:18:55.960000,2026-01-05 21:16:07.843000,30.000000,2025-12-10 21:17:59.723000,2026-01-09 21:13:03.750000,34.000000
std,NaN,NaN,8.044645,NaN,NaN,7.635468



## Contagem do Nosso Alvo (Status da Tarefa) ##


StatusTarefa
Concluída no Prazo    977
Atrasada              444
Name: count, dtype: int64

In [5]:
# Célula 3: Pré-processamento dos Dados

# 1. Criar a variável alvo (y) em formato numérico
# Usamos uma função lambda para aplicar a regra: se o status for "Atrasada", o valor é 1, senão, é 0.
df['Alvo_Atraso'] = df['StatusTarefa'].apply(lambda x: 1 if x == 'Atrasada' else 0)

# 2. Selecionar as características (features) que usaremos para a previsão
# Vamos começar com um conjunto simples de features para nosso primeiro modelo.
features = [
    'DuracaoPrevista', # A duração planejada da tarefa
    'StatusProjeto'    # O status do projeto ao qual a tarefa pertence
]

# 3. Preparar o DataFrame de features (X) e o alvo (y)
X_bruto = df[features]
y = df['Alvo_Atraso']

# 4. Transformar colunas de texto (categóricas) em colunas numéricas
# A técnica 'One-Hot Encoding' cria novas colunas para cada categoria.
# Ex: a coluna 'StatusProjeto' se transformará em 'StatusProjeto_Em Andamento', 'StatusProjeto_Concluído', etc.
X = pd.get_dummies(X_bruto, columns=['StatusProjeto'], drop_first=True)


print("## Amostra das Features (X) prontas para o modelo: ##")
display(X.head())

print("\n## Amostra do Alvo (y) pronto para o modelo: ##")
display(y.head())


## Amostra das Features (X) prontas para o modelo: ##


,DuracaoPrevista
0,19
1,23
2,9
3,24
4,17



## Amostra do Alvo (y) pronto para o modelo: ##


0    0
1    1
2    1
3    0
4    0
Name: Alvo_Atraso, dtype: int64

In [6]:
# Célula 4: Treinamento e Avaliação do Modelo

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Dividir os dados em conjuntos de treino e teste
#    Usaremos 80% para treinar e 20% para testar.
#    random_state=42 garante que a divisão seja sempre a mesma, para resultados reprodutíveis.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamanho do conjunto de treino: {len(X_train)} amostras")
print(f"Tamanho do conjunto de teste: {len(X_test)} amostras")
print("-" * 30)

# 2. Criar e treinar o modelo de IA
#    Vamos usar uma "Árvore de Decisão", um modelo clássico e fácil de interpretar.
print("Treinando o modelo de Árvore de Decisão...")
modelo = DecisionTreeClassifier(random_state=42)

# O treinamento acontece aqui! O modelo "aprende" a relação entre X_train e y_train.
modelo.fit(X_train, y_train)
print("Modelo treinado com sucesso!")
print("-" * 30)

# 3. Fazer previsões com os dados de teste (a "prova final")
print("Fazendo previsões nos dados de teste (que o modelo nunca viu)...")
previsoes = modelo.predict(X_test)

# 4. Avaliar o desempenho do modelo
print("Avaliando o desempenho do modelo...")
acuracia = accuracy_score(y_test, previsoes)
print(f"\nAcurácia do Modelo: {acuracia:.2%}")

print("\nRelatório de Classificação Detalhado:")
print(classification_report(y_test, previsoes, target_names=["No Prazo", "Atrasada"]))


Tamanho do conjunto de treino: 1136 amostras
Tamanho do conjunto de teste: 285 amostras
------------------------------
Treinando o modelo de Árvore de Decisão...
Modelo treinado com sucesso!
------------------------------
Fazendo previsões nos dados de teste (que o modelo nunca viu)...
Avaliando o desempenho do modelo...

Acurácia do Modelo: 74.04%

Relatório de Classificação Detalhado:
              precision    recall  f1-score   support

    No Prazo       0.76      0.90      0.83       196
    Atrasada       0.64      0.38      0.48        89

    accuracy                           0.74       285
   macro avg       0.70      0.64      0.65       285
weighted avg       0.73      0.74      0.72       285



In [6]:
# Célula 4: Treinamento e Avaliação do Modelo

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Dividir os dados em conjuntos de treino e teste
#    Usaremos 80% para treinar e 20% para testar.
#    random_state=42 garante que a divisão seja sempre a mesma, para resultados reprodutíveis.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamanho do conjunto de treino: {len(X_train)} amostras")
print(f"Tamanho do conjunto de teste: {len(X_test)} amostras")
print("-" * 30)

# 2. Criar e treinar o modelo de IA
#    Vamos usar uma "Árvore de Decisão", um modelo clássico e fácil de interpretar.
print("Treinando o modelo de Árvore de Decisão...")
modelo = DecisionTreeClassifier(random_state=42)

# O treinamento acontece aqui! O modelo "aprende" a relação entre X_train e y_train.
modelo.fit(X_train, y_train)
print("Modelo treinado com sucesso!")
print("-" * 30)

# 3. Fazer previsões com os dados de teste (a "prova final")
print("Fazendo previsões nos dados de teste (que o modelo nunca viu)...")
previsoes = modelo.predict(X_test)

# 4. Avaliar o desempenho do modelo
print("Avaliando o desempenho do modelo...")
acuracia = accuracy_score(y_test, previsoes)
print(f"\nAcurácia do Modelo: {acuracia:.2%}")

print("\nRelatório de Classificação Detalhado:")
print(classification_report(y_test, previsoes, target_names=["No Prazo", "Atrasada"]))

ModuleNotFoundError: No module named 'sklearn'

In [7]:
# Célula 4: Treinamento e Avaliação do Modelo

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Dividir os dados em conjuntos de treino e teste
#    Usaremos 80% para treinar e 20% para testar.
#    random_state=42 garante que a divisão seja sempre a mesma, para resultados reprodutíveis.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamanho do conjunto de treino: {len(X_train)} amostras")
print(f"Tamanho do conjunto de teste: {len(X_test)} amostras")
print("-" * 30)

# 2. Criar e treinar o modelo de IA
#    Vamos usar uma "Árvore de Decisão", um modelo clássico e fácil de interpretar.
print("Treinando o modelo de Árvore de Decisão...")
modelo = DecisionTreeClassifier(random_state=42)

# O treinamento acontece aqui! O modelo "aprende" a relação entre X_train e y_train.
modelo.fit(X_train, y_train)
print("Modelo treinado com sucesso!")
print("-" * 30)

# 3. Fazer previsões com os dados de teste (a "prova final")
print("Fazendo previsões nos dados de teste (que o modelo nunca viu)...")
previsoes = modelo.predict(X_test)

# 4. Avaliar o desempenho do modelo
print("Avaliando o desempenho do modelo...")
acuracia = accuracy_score(y_test, previsoes)
print(f"\nAcurácia do Modelo: {acuracia:.2%}")

print("\nRelatório de Classificação Detalhado:")
print(classification_report(y_test, previsoes, target_names=["No Prazo", "Atrasada"]))

Tamanho do conjunto de treino: 1136 amostras
Tamanho do conjunto de teste: 285 amostras
------------------------------
Treinando o modelo de Árvore de Decisão...
Modelo treinado com sucesso!
------------------------------
Fazendo previsões nos dados de teste (que o modelo nunca viu)...
Avaliando o desempenho do modelo...

Acurácia do Modelo: 74.04%

Relatório de Classificação Detalhado:
              precision    recall  f1-score   support

    No Prazo       0.76      0.90      0.83       196
    Atrasada       0.64      0.38      0.48        89

    accuracy                           0.74       285
   macro avg       0.70      0.64      0.65       285
weighted avg       0.73      0.74      0.72       285



In [8]:
# Célula 5: Engenharia de Features (Criando um DataFrame mais inteligente)

# Vamos começar do DataFrame original 'df' novamente
df_features = df.copy()

# 1. Transformar nosso alvo em 0 e 1
df_features['Alvo_Atraso'] = df_features['StatusTarefa'].apply(lambda x: 1 if x == 'Atrasada' else 0)

# 2. Criar features a partir das datas
df_features['MesInicioPrevisto'] = df_features['DataInicioPrevista'].dt.month
df_features['DiaDaSemanaInicioPrevisto'] = df_features['DataInicioPrevista'].dt.dayofweek # Segunda=0, Domingo=6

# 3. Criar features de interação
# A diferença entre a duração real e a prevista é uma informação valiosíssima!
df_features['DiferencaDuracao'] = df_features['DuracaoReal'] - df_features['DuracaoPrevista']

# 4. Selecionar nosso novo e melhorado conjunto de features
features_melhoradas = [
    'DuracaoPrevista',
    'StatusProjeto', # Manteremos esta
    'MesInicioPrevisto',
    'DiaDaSemanaInicioPrevisto',
    'DiferencaDuracao'
]

X_bruto_melhorado = df_features[features_melhoradas]
y_melhorado = df_features['Alvo_Atraso']

# 5. Transformar a feature de texto em numérica (One-Hot Encoding)
X_melhorado = pd.get_dummies(X_bruto_melhorado, columns=['StatusProjeto'], drop_first=True)

print("## Amostra das Novas Features (X_melhorado) prontas para o modelo: ##")
display(X_melhorado.head())

## Amostra das Novas Features (X_melhorado) prontas para o modelo: ##


,DuracaoPrevista,MesInicioPrevisto,DiaDaSemanaInicioPrevisto,DiferencaDuracao
0,19,11,4,0
1,23,10,4,1
2,9,8,0,6
3,24,8,1,-1
4,17,11,2,0


In [9]:
# Célula 6: Treinamento e Avaliação do SEGUNDO Modelo (com features melhoradas)

# As importações já foram feitas, mas é uma boa prática repeti-las
# em células de treinamento para que sejam independentes.
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

print("--- Treinando o Modelo V2 com Features Melhoradas ---")

# 1. Dividir os NOVOS dados em treino e teste
X_train_v2, X_test_v2, y_train_v2, y_test_v2 = train_test_split(
    X_melhorado, y_melhorado, test_size=0.2, random_state=42, stratify=y_melhorado
)

# 2. Criar e treinar um novo modelo
#    Vamos chamá-lo de 'modelo_2' para não confundir com o primeiro
print("Treinando o novo modelo...")
modelo_2 = DecisionTreeClassifier(random_state=42)
modelo_2.fit(X_train_v2, y_train_v2)
print("Modelo V2 treinado com sucesso!")
print("-" * 30)

# 3. Fazer e avaliar as novas previsões
print("Fazendo e avaliando as novas previsões...")
previsoes_v2 = modelo_2.predict(X_test_v2)

acuracia_v2 = accuracy_score(y_test_v2, previsoes_v2)
print(f"\nNova Acurácia do Modelo: {acuracia_v2:.2%}")

print("\nNovo Relatório de Classificação Detalhado:")
print(classification_report(y_test_v2, previsoes_v2, target_names=["No Prazo", "Atrasada"]))

--- Treinando o Modelo V2 com Features Melhoradas ---
Treinando o novo modelo...
Modelo V2 treinado com sucesso!
------------------------------
Fazendo e avaliando as novas previsões...

Nova Acurácia do Modelo: 88.42%

Novo Relatório de Classificação Detalhado:
              precision    recall  f1-score   support

    No Prazo       0.91      0.92      0.92       196
    Atrasada       0.82      0.81      0.81        89

    accuracy                           0.88       285
   macro avg       0.87      0.86      0.86       285
weighted avg       0.88      0.88      0.88       285



In [10]:
# Célula 7: Salvando o modelo e as colunas para produção

import joblib
import json

# 1. Salvar o objeto do modelo treinado em um arquivo
nome_arquivo_modelo = 'modelo_atraso_v1.joblib'
joblib.dump(modelo_2, nome_arquivo_modelo)

print(f"Modelo salvo com sucesso no arquivo: {nome_arquivo_modelo}")

# 2. Salvar a lista de colunas que o modelo espera
#    Isso é CRUCIAL para garantir que os dados da API tenham o mesmo formato
nome_arquivo_colunas = 'colunas_modelo_v1.json'
colunas = X_melhorado.columns.tolist()

with open(nome_arquivo_colunas, 'w') as f:
    json.dump(colunas, f)

print(f"Lista de colunas salva com sucesso no arquivo: {nome_arquivo_colunas}")

Modelo salvo com sucesso no arquivo: modelo_atraso_v1.joblib
Lista de colunas salva com sucesso no arquivo: colunas_modelo_v1.json
